# Q10: Based on a prediction, what are the sweet spots of the existence of crude oil?
    a. Predict the lithology of the sensor readings
    b. pick the sandstone lithology with cross over in the permeability and porosity graphs and high resistivity (higher than 10 ohm.m)
    b. visualize the sweet spots on a graph

**Purpose of the question**: interpret the analysis results to the well log plot


In [1]:
import matplotlib.backends.backend_pdf
from chart_studio import plotly as py
import plotly
import matplotlib.pyplot as plt
import numpy as np
import math
import lasio
import pandas as pd
from utils import *
import warnings

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("./Data/labeled_logs.csv")
well = df[df["WELL"] == "30/6-5"]

In [3]:
rename_columns(well)
rename_lithology(well)

In [4]:
well.columns

Index(['WELL', 'Measured Depth', 'X_LOC', 'Y_LOC', 'Z_LOC', 'GROUP',
       'FORMATION', 'Caliper', 'Shallow Resistivity',
       'Medium Deep Resistivity', 'Deep Resistivity', 'Bulk Density',
       'Gamma Ray', 'Spectra Gamma Ray', 'Neutron Porosity',
       'Photo Electric Factor', 'Compressional waves sonic', 'Self Potential',
       'Borehole Size', 'Rate of Penetration', 'Shear wave sonic',
       'Differential Caliper', 'Density Correction', 'Weight of Drilling Mud',
       'Micro Resistivity', 'Average Rate of Penetration',
       'Flushed Zone Resistivity', 'LITHOLOGY', 'CONFIDENCE'],
      dtype='object')

In [5]:
well.describe()

,Measured Depth,X_LOC,Y_LOC,Z_LOC,Caliper,Shallow Resistivity,Medium Deep Resistivity,Deep Resistivity,Bulk Density,Gamma Ray,...,Borehole Size,Rate of Penetration,Shear wave sonic,Differential Caliper,Density Correction,Weight of Drilling Mud,Micro Resistivity,Average Rate of Penetration,Flushed Zone Resistivity,CONFIDENCE
count,21636.000000,21636.000000,2.163600e+04,21636.000000,21636.000000,21543.000000,21636.000000,21636.000000,17396.000000,21636.000000,...,0.0,5428.000000,0.0,0.0,17396.000000,0.0,0.0,0.0,21543.000000,21634.000000
mean,1901.789128,497435.690403,6.728314e+06,-1876.742554,15.023619,4.406523,3.249195,2.770618,2.235966,60.485826,...,NaN,626.793661,NaN,NaN,-0.019565,NaN,NaN,NaN,4.406743,1.110798
std,950.076651,1.603602,1.772950e+00,950.036494,3.567134,15.309442,8.684753,3.984296,0.287440,28.672819,...,NaN,652.197972,NaN,NaN,0.162474,NaN,NaN,NaN,15.015536,0.400585
min,256.922400,497432.656250,6.728312e+06,-3522.748779,8.293892,0.144376,0.489001,0.408959,1.058234,13.831209,...,NaN,14.740163,NaN,NaN,-1.740271,NaN,NaN,NaN,0.151241,1.000000
25%,1079.052400,497434.187500,6.728313e+06,-2699.891785,13.062982,1.053047,1.005875,1.010638,1.975110,39.670696,...,NaN,148.326778,NaN,NaN,-0.010969,NaN,NaN,NaN,1.055680,1.000000
50%,1901.182400,497435.968750,6.728314e+06,-1876.156800,14.308678,1.827821,1.608696,1.549330,2.271871,54.092361,...,NaN,365.188965,NaN,NaN,-0.002281,NaN,NaN,NaN,1.822853,1.000000
75%,2724.984400,497437.218750,6.728316e+06,-1054.043274,18.167191,5.564113,4.066304,3.253166,2.489623,72.815086,...,NaN,943.401611,NaN,NaN,0.009558,NaN,NaN,NaN,5.616581,1.000000
max,3547.874400,497437.937500,6.728318e+06,-231.922302,24.139614,1122.207642,598.500000,128.508682,2.787034,236.665665,...,NaN,4837.952148,NaN,NaN,0.401935,NaN,NaN,NaN,1077.067993,3.000000


In [6]:
def is_crossover(row):
    if math.isnan(row["Neutron Porosity2"]) or math.isnan(row["Bulk Density2"]):
        return False
    if row["Neutron Porosity2"] + row["Bulk Density2"] > 100:
        return False
    else:
        return True

In [7]:
def translate(value, leftMin, leftMax, rightMin, rightMax):
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    valueScaled = value.apply(lambda x: float(x - leftMin) / float(leftSpan))

    return rightMin + (valueScaled * rightSpan)

In [8]:
well.head()

,WELL,Measured Depth,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,Caliper,Shallow Resistivity,Medium Deep Resistivity,...,Rate of Penetration,Shear wave sonic,Differential Caliper,Density Correction,Weight of Drilling Mud,Micro Resistivity,Average Rate of Penetration,Flushed Zone Resistivity,LITHOLOGY,CONFIDENCE
517758,30/6-5,256.9224,497437.375,6728312.0,-231.922302,NORDLAND GP.,NaN,21.149618,1.837429,1.539432,...,98.551178,NaN,NaN,NaN,NaN,NaN,NaN,1.847653,Shale,NaN
517759,30/6-5,257.0744,497437.375,6728312.0,-232.074310,NORDLAND GP.,NaN,20.023129,1.950167,1.608084,...,57.606297,NaN,NaN,NaN,NaN,NaN,NaN,2.041368,Shale,NaN
517760,30/6-5,257.2264,497437.375,6728312.0,-232.226303,NORDLAND GP.,NaN,17.702616,2.133782,1.667697,...,213.779495,NaN,NaN,NaN,NaN,NaN,NaN,2.213406,Shale,1.0
517761,30/6-5,257.3784,497437.375,6728312.0,-232.378311,NORDLAND GP.,NaN,16.079689,2.224826,1.712445,...,190.913376,NaN,NaN,NaN,NaN,NaN,NaN,2.231647,Shale,1.0
517762,30/6-5,257.5304,497437.375,6728312.0,-232.530304,NORDLAND GP.,NaN,14.906407,2.308409,1.786179,...,85.149597,NaN,NaN,NaN,NaN,NaN,NaN,2.377096,Shale,1.0


In [9]:
def get_sandstone_position_tuples(log):
    log["Neutron Porosity"] = translate(
        log["Neutron Porosity"],
        log["Neutron Porosity"].min(),
        log["Neutron Porosity"].max(),
        -0.15,
        0.45,
    )
    log["Bulk Density"] = translate(
        log["Bulk Density"],
        log["Bulk Density"].min(),
        log["Bulk Density"].max(),
        1.9,
        2.9,
    )

    log["Neutron Porosity2"] = translate(log["Neutron Porosity"], -0.15, 0.45, 0, 100)
    log["Bulk Density2"] = translate(log["Bulk Density"], 1.9, 2.9, 0, 100)

    log["Crossover"] = log.apply(is_crossover, axis=1)

    log["WATER"] = log["Deep Resistivity"].apply(lambda x: False if x > 10 else True)
    sandstone_positions = log[
        (log["LITHOLOGY"] == "Sandstone")
        & (log["WATER"] == False)
        & (log["Crossover"] == True)
    ]["Measured Depth"].tolist()

    sandstone_positions = [int(i) for i in sandstone_positions]

    sandstone_positions_tuples = []
    for i in range(len(sandstone_positions)):
        j = i + 1
        while (
            j < len(sandstone_positions)
            and sandstone_positions[j] - sandstone_positions[j - 1] == 1
        ):
            j += 1
        if j == len(sandstone_positions):
            sandstone_positions_tuples.append(
                (sandstone_positions[i], sandstone_positions[j - 1])
            )
            break
        sandstone_positions_tuples.append(
            (sandstone_positions[i], sandstone_positions[j - 1])
        )
        i = j

    return sandstone_positions_tuples

In [10]:
def add_graph(
    ax,
    x,
    y,
    label,
    color,
    xlim_min,
    xlim_max,
    spines,
    spines_value,
    minor_ticks=None,
    major_ticks=None,
    new_grid=False,
    log_scale=False,
):
    ax_new = ax.twiny()
    if log_scale:
        ax_new.set_xscale("log")
    ax_new.set_xlim(xlim_min, xlim_max)
    ax_new.set_xlabel(label, color=color, fontsize=12)
    ax_new.plot(
        x,
        y,
        spines,
        color=color,
        label=label,
        linewidth=1,
    )
    ax_new.tick_params(axis="x", colors=color)
    ax_new.spines["top"].set_position(("outward", spines_value))
    if type(minor_ticks) != type(None) and type(major_ticks) != type(None):
        ax_new.set_xticks(major_ticks)
        ax_new.set_xticks(minor_ticks, minor=True)
    if new_grid:
        ax_new.grid(which="minor", alpha=0.5)
        ax_new.grid(which="major", alpha=1)

In [11]:
def draw_first_track(ax, log):
    ax[0].set_ylabel("Depth [m]", color="Black", fontsize=15)
    add_graph(
        ax[0],
        log["Gamma Ray"],
        log["Measured Depth"],
        "Gamma Ray [GAPI]",
        "Green",
        0,
        200,
        "-",
        50,
        minor_ticks=np.arange(0, 201, 20),
        major_ticks=np.arange(0, 201, 100),
        new_grid=True,
    )
    add_graph(
        ax[0],
        log["Caliper"],
        log["Measured Depth"],
        "Caliper [in]",
        "Red",
        6,
        16,
        "--",
        10,
        minor_ticks=np.arange(6, 17, 1),
        major_ticks=np.arange(6, 17, 5),
    )
    

In [12]:
def draw_second_track(ax, log):
    add_graph(
        ax[1],
        log["Medium Deep Resistivity"],
        log["Measured Depth"],
        "Medium Deep Resistivity [ohm.m]",
        "Red",
        0.2,
        200,
        "--",
        90,
        log_scale=True,
        new_grid=True,
    )
    add_graph(
        ax[1],
        log["Deep Resistivity"],
        log["Measured Depth"],
        "Deep Resistivity [ohm.m]",
        "Black",
        0.2,
        200,
        "-",
        10,
        log_scale=True,
    )
    add_graph(
        ax[1],
        log["Shallow Resistivity"],
        log["Measured Depth"],
        "Shallow Resistivity [ohm.m]",
        "Blue",
        0.2,
        200,
        "--",
        50,
        log_scale=True,
    )

In [13]:
def draw_third_track(ax, log):
    add_graph(
        ax[2],
        log["Neutron Porosity"],
        log["Measured Depth"],
        "Neutron Porosity [decp]",
        "Blue",
        0.45,
        -0.15,
        "--",
        50,
        minor_ticks=np.arange(-0.15, 0.45, 0.06),
        major_ticks=np.arange(-0.15, 0.45, 0.3),
        new_grid=True,
    )

    add_graph(
        ax[2],
        log["Bulk Density"],
        log["Measured Depth"],
        "Bulk Density [g/cc]",
        "Red",
        1.9,
        2.9,
        "-",
        10,
        minor_ticks=np.arange(1.9, 2.9, 0.1),
        major_ticks=np.arange(1.9, 2.9, 0.5),
    )
    

In [14]:
def highlight_reservoir(ax, sandstone_positions_tuples, top, bottom):
    for start, end in sandstone_positions_tuples:
        s = start if start > top else top
        e = end if end < bottom else bottom
        ax[0].axhspan(s, e, color="yellow", alpha=0.5)
        ax[1].axhspan(s, e, color="yellow", alpha=0.5)
        ax[2].axhspan(s, e, color="yellow", alpha=0.5)
        if e == bottom:
            break

In [15]:
def plot_3_tracks(log):
    log = log.copy()
    sandstone_positions_tuples = get_sandstone_position_tuples(log)
    pdf = matplotlib.backends.backend_pdf.PdfPages("wireline_log.pdf")
    o_top = log.iloc[0]["Measured Depth"]
    o_bottom = log.iloc[-1]["Measured Depth"]

    # for every 100 readings, plot the log in a new page
    i = o_top
    while i < o_bottom:
        top = i
        bottom = i + 100
        i = i + 100
        fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 25), sharey=True)
        fig.suptitle("Wireline Log", fontsize=20)
        fig.subplots_adjust(top=0.88)

        for axes in ax:
            axes.set_ylim(bottom, top)
            depth_major_ticks = np.arange(top, bottom, 50)
            depth_minor_ticks = np.arange(top, bottom, 10)
            axes.set_yticks(depth_major_ticks)
            axes.set_yticks(depth_minor_ticks, minor=True)
            axes.get_xaxis().set_visible(False)
            axes.grid(which="minor", axis="y", alpha=0.5)
            axes.grid(which="major", axis="y", alpha=1)

        draw_first_track(ax, log)
        draw_second_track(ax, log)
        draw_third_track(ax, log)

        highlight_reservoir(ax, sandstone_positions_tuples, top, bottom)

        pdf.savefig(fig)
        plt.close()
    pdf.close()


In [16]:
plot_3_tracks(well)
